In [1]:
import pandas as pd

In [2]:
bat_df = pd.read_csv('../Datasets/batter_stats.csv')
pitch_df = pd.read_csv('../Datasets/pitcher_stats.csv')
lineups = pd.read_csv('../Datasets/Lineups_2021_06_13.csv')

In [3]:
# The dots in names like J.T. were inconsistent across datasets
bat_df.loc[:,'Name'] = bat_df['Name'].str.replace('.', '')
pitch_df.loc[:,'Name'] = pitch_df['Name'].str.replace('.', '')
lineups.loc[:,' player name'] = lineups[' player name'].str.replace('.', '')

In [4]:
bats_r = bat_df[bat_df['bats'] == 'R']
bats_l = bat_df[bat_df['bats'] == 'L']

In [5]:
# Making name column consistent
lineups.columns = lineups.columns.str.replace(' player name', 'Name')

In [6]:
# Individual name that was giving trouble
lineups.loc[lineups['Name'] == 'Jonathan Gray', 'Name'] = 'Jon Gray'

In [7]:
# Creates a list of teams with no repeats
# The order needs to be kept from the dataframe because 
# opposing teams are paired
team_list = []
for team in lineups['team code']:
    if team not in team_list:
        team_list.append(team)
# Creating a dictionary of every team, and the team they face
matchups = {}
for i in range(1, len(team_list), 2):
    matchups[team_list[i]] = team_list[i-1]
    matchups[team_list[i-1]] = team_list[i]

In [8]:
lineups

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position
0,NYY,6/13/2021,1,518934,DJ LeMahieu,1,Y,1B
1,NYY,6/13/2021,1,650402,Gleyber Torres,2,Y,SS
2,NYY,6/13/2021,1,596059,Rougned Odor,3,Y,2B
3,NYY,6/13/2021,1,570482,Giovanny Urshela,4,Y,3B
4,NYY,6/13/2021,1,596142,Gary Sanchez,5,Y,C
...,...,...,...,...,...,...,...,...
277,CHC,6/13/2021,1,664023,Ian Happ,5,Y,CF
278,CHC,6/13/2021,1,621550,Patrick Wisdom,6,Y,3B
279,CHC,6/13/2021,1,519299,Eric Sogard,7,Y,2B
280,CHC,6/13/2021,1,642727,Sergio Alcantara,8,Y,SS


In [9]:
batters = lineups.loc[(lineups[' batting order'] != 'SP') & (lineups[' game_number'] == 1)]

In [10]:
pitch_df.drop(456, inplace = True)

In [11]:
matchups

{'PHI': 'NYY',
 'NYY': 'PHI',
 'WAS': 'SF',
 'SF': 'WAS',
 'CLE': 'SEA',
 'SEA': 'CLE',
 'MIA': 'ATL',
 'ATL': 'MIA',
 'CIN': 'COL',
 'COL': 'CIN',
 'TB': 'BAL',
 'BAL': 'TB',
 'BOS': 'TOR',
 'TOR': 'BOS',
 'NYM': 'SD',
 'SD': 'NYM',
 'DET': 'CHW',
 'CHW': 'DET',
 'MIN': 'HOU',
 'HOU': 'MIN',
 'MIL': 'PIT',
 'PIT': 'MIL',
 'OAK': 'KC',
 'KC': 'OAK',
 'ARI': 'LAA',
 'LAA': 'ARI',
 'LAD': 'TEX',
 'TEX': 'LAD',
 'CHC': 'STL',
 'STL': 'CHC'}

In [12]:
pitch_df.loc[pitch_df['Name']== 'Chase DeJong', 'throws']

Series([], Name: throws, dtype: object)

In [13]:
opp_pitch_hand = []
opp_pitcher = []
del_indexes = []
keep_pitch = []
for i, row in batters.iterrows():
    opp_pitch = lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                            & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values[0]
    if len(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values) > 0:
            opp_pitch_hand.append(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values[0])
            opp_pitcher.append(opp_pitch)
            keep_pitch.append(opp_pitch)
    else:
        del_indexes.append(i)
        
batters.drop(index = del_indexes, inplace = True)
batters['opp_pitch_hand'] = opp_pitch_hand
batters['opp_pitch'] = opp_pitcher


/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-13-9eb6f881f0bf>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters['opp_pitch_hand'] = opp_pitch_hand
<ipython-input-13-9eb6f881f0bf>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [14]:
del_indexes

[74, 75, 76, 77, 78, 79, 80, 81]

In [15]:
bats = []
k_pct = []
bb_pct = []
bb_k = []
bat_avg = []
obp = []
slg = []
ops = []
iso = []
babip = []
wrc = []
wraa = []
woba = []
wrc_plus = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    if row['opp_pitch_hand'] == 'R':
        batter_df = bat_df.loc[(bat_df['pitch']== 'R')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'L']
    if row['opp_pitch_hand'] == 'L':
        batter_df = bat_df.loc[(bat_df['pitch']== 'L')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'R']
    if len(batter_df) == 0:
        for col in batter_df.columns:
            batter_df[col] = [float('NaN')]
    bats.append(batter_df['bats'].values[0])
    k_pct.append(batter_df['K%'].values[0])
    bb_pct.append(batter_df['BB%'].values[0])
    bb_k.append(batter_df['BB/K'].values[0])
    bat_avg.append(batter_df['AVG'].values[0])
    obp.append(batter_df['OBP'].values[0])
    slg.append(batter_df['SLG'].values[0])
    ops.append(batter_df['OPS'].values[0])
    iso.append(batter_df['ISO'].values[0])
    babip.append(batter_df['BABIP'].values[0])
    wrc.append(batter_df['wRC'].values[0])
    wraa.append(batter_df['wRAA'].values[0])
    woba.append(batter_df['wOBA'].values[0])
    wrc_plus.append(batter_df['wRC+'].values[0])
batters.loc[:,'bats'] = bats
batters.loc[:,'k_pct'] = k_pct
batters.loc[:,'bb_pct'] = bb_pct
batters.loc[:,'bb_k'] = bb_k
batters.loc[:,'bat_avg'] = bat_avg
batters.loc[:,'obp'] = obp
batters.loc[:,'slg'] = slg
batters.loc[:,'ops'] = ops
batters.loc[:,'iso'] = iso
batters.loc[:,'babip'] = babip
batters.loc[:,'wrc'] = wrc
batters.loc[:,'wraa'] = wraa
batters.loc[:,'woba'] = woba
batters.loc[:,'wrc_plus'] = wrc_plus

<ipython-input-15-0499cefeeb18>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batter_df[col] = [float('NaN')]
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [16]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,bat_avg,obp,slg,ops,iso,babip,wrc,wraa,woba,wrc_plus
0,NYY,6/13/2021,1,518934,DJ LeMahieu,1,Y,1B,R,Aaron Nola,...,0.311594,0.384615,0.471014,0.855630,0.159420,0.339367,52.677512,14.763552,0.370998,138.569733
1,NYY,6/13/2021,1,650402,Gleyber Torres,2,Y,SS,R,Aaron Nola,...,0.252336,0.352227,0.359813,0.712040,0.107477,0.308642,31.166926,1.296157,0.320848,104.400282
2,NYY,6/13/2021,1,596059,Rougned Odor,3,Y,2B,R,Aaron Nola,...,0.191781,0.259259,0.424658,0.683917,0.232877,0.195652,16.412341,-3.342485,0.290395,83.343937
3,NYY,6/13/2021,1,570482,Giovanny Urshela,4,Y,3B,R,Aaron Nola,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NYY,6/13/2021,1,596142,Gary Sanchez,5,Y,C,R,Aaron Nola,...,0.171717,0.277533,0.353535,0.631068,0.181818,0.210526,20.999703,-6.691094,0.280507,76.440954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,CHC,6/13/2021,1,575929,Willson Contreras,4,Y,C,R,Carlos Martinez,...,0.230769,0.354515,0.392713,0.747228,0.161943,0.301282,41.119654,4.685667,0.333730,110.204660
277,CHC,6/13/2021,1,664023,Ian Happ,5,Y,CF,R,Carlos Martinez,...,0.236948,0.347079,0.485944,0.833023,0.248996,0.275641,46.007419,10.541919,0.358711,126.632292
278,CHC,6/13/2021,1,621550,Patrick Wisdom,6,Y,3B,R,Carlos Martinez,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,CHC,6/13/2021,1,519299,Eric Sogard,7,Y,2B,R,Carlos Martinez,...,0.235294,0.288770,0.323529,0.612299,0.088235,0.265734,15.437975,-7.383582,0.268490,66.367489


In [17]:
k_9 = []
bb_9 = []
k_bb = []
hr_9 = []
k_pct = []
bb_pct = []
k_bb_pct = []
bat_avg = []
whip = []
babip = []
lob_pct = []
fip = []
xfip = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    pitcher_df = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== row['bats'])&(pitch_df['Name']==row['opp_pitch']),:]
    if len(pitcher_df) == 0:
        for col in pitcher_df.columns:
            pitcher_df[col] = [float('NaN')]
    k_9.append(pitcher_df['K/9'].values[0])
    bb_9.append(pitcher_df['BB/9'].values[0])
    k_bb.append(pitcher_df['K/BB'].values[0])
    hr_9.append(pitcher_df['HR/9'].values[0])
    k_pct.append(pitcher_df['K%'].values[0])
    bb_pct.append(pitcher_df['BB%'].values[0])
    k_bb_pct.append(pitcher_df['K-BB%'].values[0])
    bat_avg.append(pitcher_df['AVG'].values[0])
    whip.append(pitcher_df['WHIP'].values[0])
    babip.append(pitcher_df['BABIP'].values[0])
    lob_pct.append(pitcher_df['LOB%'].values[0])
    fip.append(pitcher_df['FIP'].values[0])
    xfip.append(pitcher_df['xFIP'].values[0])
batters.loc[:,'opp_k/9'] = k_9
batters.loc[:,'opp_bb/9'] = k_9
batters.loc[:,'opp_k/bb'] = k_9
batters.loc[:,'opp_hr/9'] = k_9
batters.loc[:,'opp_k_pct'] = k_9
batters.loc[:,'opp_bb_pct'] = k_9
batters.loc[:,'opp_k_bb_pct'] = k_9
batters.loc[:,'opp_avg'] = k_9
batters.loc[:,'opp_whip'] = k_9
batters.loc[:,'opp_babip'] = k_9
batters.loc[:,'opp_lob_pct'] = k_9
batters.loc[:,'opp_fip'] = k_9
batters.loc[:,'opp_xfip'] = k_9

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [18]:
pitchers = lineups.loc[(lineups[' batting order'] == 'SP')]

In [19]:
pitchers = pitchers[pitchers['Name'].isin(keep_pitch)]

In [20]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,opp_hr/9,opp_k_pct,opp_bb_pct,opp_k_bb_pct,opp_avg,opp_whip,opp_babip,opp_lob_pct,opp_fip,opp_xfip
0,NYY,6/13/2021,1,518934,DJ LeMahieu,1,Y,1B,R,Aaron Nola,...,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013
1,NYY,6/13/2021,1,650402,Gleyber Torres,2,Y,SS,R,Aaron Nola,...,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013
2,NYY,6/13/2021,1,596059,Rougned Odor,3,Y,2B,R,Aaron Nola,...,10.451613,10.451613,10.451613,10.451613,10.451613,10.451613,10.451613,10.451613,10.451613,10.451613
3,NYY,6/13/2021,1,570482,Giovanny Urshela,4,Y,3B,R,Aaron Nola,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NYY,6/13/2021,1,596142,Gary Sanchez,5,Y,C,R,Aaron Nola,...,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013,11.523013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,CHC,6/13/2021,1,575929,Willson Contreras,4,Y,C,R,Carlos Martinez,...,6.171429,6.171429,6.171429,6.171429,6.171429,6.171429,6.171429,6.171429,6.171429,6.171429
277,CHC,6/13/2021,1,664023,Ian Happ,5,Y,CF,R,Carlos Martinez,...,5.538462,5.538462,5.538462,5.538462,5.538462,5.538462,5.538462,5.538462,5.538462,5.538462
278,CHC,6/13/2021,1,621550,Patrick Wisdom,6,Y,3B,R,Carlos Martinez,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,CHC,6/13/2021,1,519299,Eric Sogard,7,Y,2B,R,Carlos Martinez,...,5.538462,5.538462,5.538462,5.538462,5.538462,5.538462,5.538462,5.538462,5.538462,5.538462


In [21]:
num_L_bats = []
num_R_bats = []
for i, row in pitchers.iterrows():
    opp_lineup = batters.loc[(batters['team code'] == matchups[row['team code']]),:]
    find_batters = []
    # There are batters whos data was null due to lack of plate appearances but handedness is known
    for j, row2 in opp_lineup.iterrows():
        if row2['bats'] != 'R' and row2['bats'] != 'L':
            find_bats = bat_df.loc[bat_df['Name'] == row2['Name'],:]
            if len(find_bats) > 0:
                find_batters.append((j, find_bats['bats'].values[0]))
            # If batter handedness is unknown, right is more likely
            if len(find_bats) == 0:
                find_batters.append((j, 'R'))
    for handedness in find_batters:
        opp_lineup.loc[handedness[0], 'bats'] = handedness[1]
    try:
        num_L_bats.append(opp_lineup['bats'].value_counts()['L'])
    except:
        num_L_bats.append(0)
    try:
        num_R_bats.append(opp_lineup['bats'].value_counts()['R'])
    except:
        num_R_bats.append(0)
pitchers['num_L_bats'] = num_L_bats
pitchers['num_R_bats'] = num_R_bats
is_nl = []
for i, row in pitchers.iterrows():
    if row['num_L_bats'] + row['num_R_bats'] == 9:
        is_nl.append(0)
    else:
        is_nl.append(1)
pitchers['is_nl'] = is_nl

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [22]:
k_9_l = []
bb_9_l = []
k_bb_l = []
hr_9_l = []
k_pct_l = []
bb_pct_l = []
k_bb_pct_l = []
avg_l = []
whip_l = []
babip_l = []
lob_pct_l = []
fip_l = []
xfip_l = []
k_9_r = []
bb_9_r = []
k_bb_r = []
hr_9_r = []
k_pct_r = []
bb_pct_r = []
k_bb_pct_r = []
avg_r = []
whip_r = []
babip_r = []
lob_pct_r = []
fip_r = []
xfip_r = []
# Engineering features
k_9_l_tot = []
bb_9_l_tot = []
k_bb_l_tot = []
hr_9_l_tot = []
k_pct_l_tot = []
bb_pct_l_tot = []
k_bb_pct_l_tot = []
avg_l_tot = []
whip_l_tot = []
babip_l_tot = []
lob_pct_l_tot = []
fip_l_tot = []
xfip_l_tot = []
k_9_r_tot = []
bb_9_r_tot = []
k_bb_r_tot = []
hr_9_r_tot = []
k_pct_r_tot = []
bb_pct_r_tot = []
k_bb_pct_r_tot = []
avg_r_tot = []
whip_r_tot = []
babip_r_tot = []
lob_pct_r_tot = []
fip_r_tot = []
xfip_r_tot = []
for i, row in pitchers.iterrows():
    # Pulling stats of pitcher against lefty hitters
    pitcher_df_l = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'L')&(pitch_df['Name']==row['Name']),:]
    if len(pitcher_df_l) == 0:
        for col in pitcher_df_l.columns:
            pitcher_df_l[col] = [float('NaN')]
    k_9_l.append(pitcher_df_l['K/9'].values[0])
    bb_9_l.append(pitcher_df_l['BB/9'].values[0])
    k_bb_l.append(pitcher_df_l['K/BB'].values[0])
    hr_9_l.append(pitcher_df_l['HR/9'].values[0])
    k_pct_l.append(pitcher_df_l['K%'].values[0])
    bb_pct_l.append(pitcher_df_l['BB%'].values[0])
    k_bb_pct_l.append(pitcher_df_l['K-BB%'].values[0])
    avg_l.append(pitcher_df_l['AVG'].values[0])
    whip_l.append(pitcher_df_l['WHIP'].values[0])
    babip_l.append(pitcher_df_l['BABIP'].values[0])
    lob_pct_l.append(pitcher_df_l['LOB%'].values[0])
    fip_l.append(pitcher_df_l['FIP'].values[0])
    xfip_l.append(pitcher_df_l['xFIP'].values[0])
    # Feature Engineering
    k_9_l_tot.append(pitcher_df_l['K/9'].values[0] * row['num_L_bats'])
    bb_9_l_tot.append(pitcher_df_l['BB/9'].values[0] * row['num_L_bats'])
    k_bb_l_tot.append(pitcher_df_l['K/BB'].values[0] * row['num_L_bats'])
    hr_9_l_tot.append(pitcher_df_l['HR/9'].values[0] * row['num_L_bats'])
    k_pct_l_tot.append(pitcher_df_l['K%'].values[0] * row['num_L_bats'])
    bb_pct_l_tot.append(pitcher_df_l['BB%'].values[0] * row['num_L_bats'])
    k_bb_pct_l_tot.append(pitcher_df_l['K-BB%'].values[0] * row['num_L_bats'])
    avg_l_tot.append(pitcher_df_l['AVG'].values[0] * row['num_L_bats'])
    whip_l_tot.append(pitcher_df_l['WHIP'].values[0] * row['num_L_bats'])
    babip_l_tot.append(pitcher_df_l['BABIP'].values[0] * row['num_L_bats'])
    lob_pct_l_tot.append(pitcher_df_l['LOB%'].values[0] * row['num_L_bats'])
    fip_l_tot.append(pitcher_df_l['FIP'].values[0] * row['num_L_bats'])
    xfip_l_tot.append(pitcher_df_l['xFIP'].values[0] * row['num_L_bats'])
    # Pulling stats against righty hitters
    pitcher_df_r = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'R')&(pitch_df['Name']== row['Name']),:]
    if len(pitcher_df_r) == 0:
        for col in pitcher_df_r.columns:
            pitcher_df_r[col] = [float('NaN')]
    k_9_r.append(pitcher_df_r['K/9'].values[0])
    bb_9_r.append(pitcher_df_r['BB/9'].values[0])
    k_bb_r.append(pitcher_df_r['K/BB'].values[0])
    hr_9_r.append(pitcher_df_r['HR/9'].values[0])
    k_pct_r.append(pitcher_df_r['K%'].values[0])
    bb_pct_r.append(pitcher_df_r['BB%'].values[0])
    k_bb_pct_r.append(pitcher_df_r['K-BB%'].values[0])
    avg_r.append(pitcher_df_r['AVG'].values[0])
    whip_r.append(pitcher_df_r['WHIP'].values[0])
    babip_r.append(pitcher_df_r['BABIP'].values[0])
    lob_pct_r.append(pitcher_df_r['LOB%'].values[0])
    fip_r.append(pitcher_df_r['FIP'].values[0])
    xfip_r.append(pitcher_df_r['xFIP'].values[0])
    # Feature Engineering
    k_9_r_tot.append(pitcher_df_r['K/9'].values[0] * row['num_R_bats'])
    bb_9_r_tot.append(pitcher_df_r['BB/9'].values[0] * row['num_R_bats'])
    k_bb_r_tot.append(pitcher_df_r['K/BB'].values[0] * row['num_R_bats'])
    hr_9_r_tot.append(pitcher_df_r['HR/9'].values[0] * row['num_R_bats'])
    k_pct_r_tot.append(pitcher_df_r['K%'].values[0] * row['num_R_bats'])
    bb_pct_r_tot.append(pitcher_df_r['BB%'].values[0] * row['num_R_bats'])
    k_bb_pct_r_tot.append(pitcher_df_r['K-BB%'].values[0] * row['num_R_bats'])
    avg_r_tot.append(pitcher_df_r['AVG'].values[0] * row['num_R_bats'])
    whip_r_tot.append(pitcher_df_r['WHIP'].values[0] * row['num_R_bats'])
    babip_r_tot.append(pitcher_df_r['BABIP'].values[0] * row['num_R_bats'])
    lob_pct_r_tot.append(pitcher_df_r['LOB%'].values[0] * row['num_R_bats'])
    fip_r_tot.append(pitcher_df_r['FIP'].values[0] * row['num_R_bats'])
    xfip_r_tot.append(pitcher_df_r['xFIP'].values[0] * row['num_R_bats'])
pitchers['k_9_l'] = k_9_l
pitchers['bb_9_l'] = bb_9_l
pitchers['k_bb_l'] = k_bb_l
pitchers['hr_9_l'] = hr_9_l
pitchers['k_pct_l'] = k_pct_l
pitchers['bb_pct_l'] = bb_pct_l
pitchers['k_bb_pct_l'] = k_bb_pct_l
pitchers['bat_avg_l'] = avg_l
pitchers['whip_l'] = whip_l
pitchers['babip_l'] = babip_l
pitchers['lob_pct_l'] = lob_pct_l
pitchers['fip_l'] = fip_l
pitchers['xfip_l'] = xfip_l
# Right handed batter stats
pitchers['k_9_r'] = k_9_r
pitchers['bb_9_r'] = bb_9_r
pitchers['k_bb_r'] = k_bb_r
pitchers['hr_9_r'] = hr_9_r
pitchers['k_pct_r'] = k_pct_r
pitchers['bb_pct_r'] = bb_pct_r
pitchers['k_bb_pct_r'] = k_bb_pct_r
pitchers['bat_avg_r'] = avg_r
pitchers['whip_r'] = whip_r
pitchers['babip_r'] = babip_r
pitchers['lob_pct_r'] = lob_pct_r
pitchers['fip_r'] = fip_r
pitchers['xfip_r'] = xfip_r
# feature engineered left handed batter stats
pitchers['k_9_l_tot'] = k_9_l_tot
pitchers['bb_9_l_tot'] = bb_9_l_tot
pitchers['k_bb_l_tot'] = k_bb_l_tot
pitchers['hr_9_l_tot'] = hr_9_l_tot
pitchers['k_pct_l_tot'] = k_pct_l_tot
pitchers['bb_pct_l_tot'] = bb_pct_l_tot
pitchers['k_bb_pct_l_tot'] = k_bb_pct_l_tot
pitchers['bat_avg_l_tot'] = avg_l_tot
pitchers['whip_l_tot'] = whip_l_tot
pitchers['babip_l_tot'] = babip_l_tot
pitchers['lob_pct_l_tot'] = lob_pct_l_tot
pitchers['fip_l_tot'] = fip_l_tot
pitchers['xfip_l_tot'] = xfip_l_tot
# feature engineered righ handed batter stats
pitchers['k_9_r_tot'] = k_9_r_tot
pitchers['bb_9_r_tot'] = bb_9_r_tot
pitchers['k_bb_r_tot'] = k_bb_r_tot
pitchers['hr_9_r_tot'] = hr_9_r_tot
pitchers['k_pct_r_tot'] = k_pct_r_tot
pitchers['bb_pct_r_tot'] = bb_pct_r_tot
pitchers['k_bb_pct_r_tot'] = k_bb_pct_r_tot
pitchers['bat_avg_r_tot'] = avg_r_tot
pitchers['whip_r_tot'] = whip_r_tot
pitchers['babip_r_tot'] = babip_r_tot
pitchers['lob_pct_r_tot'] = lob_pct_r_tot
pitchers['fip_r_tot'] = fip_r_tot
pitchers['xfip_r_tot'] = xfip_r_tot

In [23]:
pitchers

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,hr_9_r_tot,k_pct_r_tot,bb_pct_r_tot,k_bb_pct_r_tot,bat_avg_r_tot,whip_r_tot,babip_r_tot,lob_pct_r_tot,fip_r_tot,xfip_r_tot
8,NYY,6/13/2021,1,593334,Domingo German,SP,Y,NaN,2,6,...,11.045455,1.551724,0.258621,1.293103,1.200000,5.522727,1.297297,5.510204,26.189111,22.730933
17,PHI,6/13/2021,1,605400,Aaron Nola,SP,Y,NaN,2,6,...,6.100418,1.974194,0.270968,1.703226,1.294521,5.799163,1.780220,4.985163,16.275024,16.352439
26,SF,6/13/2021,1,456501,Johnny Cueto,SP,Y,NaN,3,5,...,5.494186,1.043478,0.239130,0.804348,1.232228,5.494186,1.406250,3.146853,19.335912,20.025662
35,WAS,6/13/2021,1,605452,Joe Ross,SP,Y,NaN,5,3,...,5.845361,0.572519,0.137405,0.435114,0.719008,3.247423,0.717391,2.061069,15.165315,12.390471
45,SEA,6/13/2021,1,669302,Logan Gilbert,SP,Y,NaN,4,5,...,6.750000,1.612903,0.322581,1.290323,1.111111,6.000000,1.562500,4.069767,19.437896,20.161042
55,CLE,6/13/2021,1,669456,Shane Bieber,SP,Y,NaN,4,5,...,6.086066,1.859375,0.312500,1.546875,0.976431,4.795082,1.398810,4.411765,14.036401,11.524234
64,ATL,6/13/2021,1,592767,Drew Smyly,SP,Y,NaN,1,7,...,13.768212,1.757848,0.627803,1.130045,1.732323,9.596027,1.970370,5.512821,35.153210,30.940599
73,MIA,6/13/2021,1,641154,Pablo Lopez,SP,Y,NaN,3,5,...,6.567568,1.625514,0.432099,1.193416,1.013825,5.270270,1.315789,3.838951,17.837156,15.172177
82,COL,6/13/2021,1,622608,Antonio Senzatela,SP,Y,NaN,4,4,...,5.586207,0.661290,0.193548,0.467742,1.052632,4.965517,1.127072,2.732919,18.603885,17.451413
101,BAL,6/13/2021,1,669145,Bruce Zimmermann,SP,Y,NaN,2,7,...,15.514925,1.220513,0.466667,0.753846,1.988636,9.873134,2.052632,5.697674,41.751511,35.436221


**Data Description:** This worksheet provides a framework for creating dataframes for each days' players and stats. I will combine these dataframes across many days to get my final training data. These data frames have all of the stats that I could potentially be interested in, the features will be reduced for modeling as there is a significant amount of correlation between many of the features. The batters dataframe includes the batters stats against pitchers who throw with the same hand as the opposing pitcher, and includes the stats of how the opposing pitcher performs against batters with the same handedness of the batter. The pitchers dataframe contains each pitcher and their stats against both right handed and left handed batters. It also contains the number of right and left handed batters in the opposing lineup. I engineered features that are the pitchers stats vs the handedness of a batter multiplied by the number of those handed batters in the lineup. My final feature is is_nl. This is 1 if the lineup they are facing includes a pitcher and 0 if not.

**Data Dictionary:**   
team code: The team the player plays for.  
Name: The players name  
Batting order: The order the batters hit in the lineup. Is SP if player is a pitcher.  
opp_pitch_hand(batters): The hand that the opposing pitcher throws with  
opp_pitch(batters):The name of the opposing pitcher. 
opp_x(batters): This is the format for column names of stats that the opposing pitcher has against batters of the same handedness of the batter.  
(batters): There are also many columns of stats for how the batter performs against pitchers with the same handedness of the opposing pitcher.    
num_L_bats(pitchers): The number of left handed batters in the opposing lineup.  
num_R_bats(pitchers): The number of right handed batters in the opposing lineup.  
x_(l/r) (pitchers): stats describing how the pitcher performs against either left or right handed batters.    
x_(l/r)_tot (pitchers): The stats of how a pitcher performs against both hands multiplied by the number of those handed batters in the lineup.

In [24]:
scores = pd.read_csv('../datasets/6_13_target.csv', delimiter = ';')
scores.head()

,Date,GID,MLB_ID,Name,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,20210613,129f,605452,"Ross, Joe",1.0,9,1.0,37.00,7700.0,WAS,v sfo,NaN,5,0
1,20210613,130m,605400,"Nola, Aaron",1.0,9,1.0,36.85,9600.0,PHI,v nyy,NaN,7,0
2,20210613,127s,607074,"Rodon, Carlos",1.0,0,1.0,33.95,10200.0,CHW,@ det,NaN,4,1
3,20210613,132c,605200,"Davies, Zach",1.0,9,NaN,28.60,NaN,CHC,v stl,NaN,2,0
4,20210613,151k,663978,"Paddack, Chris",1.0,9,1.0,27.90,9400.0,SDG,@ nym,NaN,7,3


In [25]:
def clean_names(name):
    name = name.replace(',','')
    name = name.replace('.','')
    name = name.replace('-hitter','')
    name = name.replace('-pitcher','')
    names = name.split()
    first = names.pop(-1)
    names.insert(0, first)
    name = ' '.join(names)
    name = name.replace('James Happ', 'JA Happ')
    return name

In [26]:
scores['Name'] = [clean_names(name) for name in scores['Name']]

In [27]:
pitch_s = scores[scores['DK posn'] == 1]
hit_s = scores[scores['DK posn'] != 1]

In [28]:
pitch_final = pd.merge(left = pitchers, right = pitch_s, how = 'inner')

In [29]:
bat_final = pd.merge(left = batters, right = hit_s,
         how = 'inner').dropna(how = 'any', subset = ['bats', 'DK pts', 'Starter'])

In [30]:
bat_final = bat_final[bat_final['dblhdr'].isna()]

In [31]:
bat_final.to_csv('../merged_data/hitters_6_13.csv')
pitch_final.to_csv('../merged_data/pitchers_6_13.csv')

In [32]:
bat_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,NYY,6/13/2021,1,518934,DJ LeMahieu,1,Y,1B,R,Aaron Nola,...,1.0,1,34.0,6.0,4100.0,NYY,@ phi,NaN,0,7
1,NYY,6/13/2021,1,650402,Gleyber Torres,2,Y,SS,R,Aaron Nola,...,1.0,2,6.0,5.0,4500.0,NYY,@ phi,NaN,0,7
2,NYY,6/13/2021,1,596059,Rougned Odor,3,Y,2B,R,Aaron Nola,...,1.0,3,4.0,5.0,3200.0,NYY,@ phi,NaN,0,7
4,NYY,6/13/2021,1,596142,Gary Sanchez,5,Y,C,R,Aaron Nola,...,1.0,5,2.0,0.0,4600.0,NYY,@ phi,NaN,0,7
5,NYY,6/13/2021,1,609280,Miguel Andujar,6,Y,LF,R,Aaron Nola,...,1.0,6,7.0,0.0,3000.0,NYY,@ phi,NaN,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,CHC,6/13/2021,1,592178,Kris Bryant,2,Y,RF,R,Carlos Martinez,...,1.0,2,NaN,3.0,NaN,CHC,v stl,NaN,2,0
225,CHC,6/13/2021,1,519203,Anthony Rizzo,3,Y,1B,R,Carlos Martinez,...,1.0,3,NaN,5.0,NaN,CHC,v stl,NaN,2,0
226,CHC,6/13/2021,1,575929,Willson Contreras,4,Y,C,R,Carlos Martinez,...,1.0,4,NaN,0.0,NaN,CHC,v stl,NaN,2,0
227,CHC,6/13/2021,1,664023,Ian Happ,5,Y,CF,R,Carlos Martinez,...,1.0,5,NaN,0.0,NaN,CHC,v stl,NaN,2,0


In [33]:
pitch_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,NYY,6/13/2021,1,593334,Domingo German,SP,Y,NaN,2,6,...,1.0,9,1.0,-6.85,8900.0,NYY,@ phi,NaN,0,7
1,PHI,6/13/2021,1,605400,Aaron Nola,SP,Y,NaN,2,6,...,1.0,9,1.0,36.85,9600.0,PHI,v nyy,NaN,7,0
2,SF,6/13/2021,1,456501,Johnny Cueto,SP,Y,NaN,3,5,...,1.0,9,1.0,0.80,8100.0,SFO,@ was,NaN,0,5
3,WAS,6/13/2021,1,605452,Joe Ross,SP,Y,NaN,5,3,...,1.0,9,1.0,37.00,7700.0,WAS,v sfo,NaN,5,0
4,SEA,6/13/2021,1,669302,Logan Gilbert,SP,Y,NaN,4,5,...,1.0,0,1.0,26.00,6700.0,SEA,@ cle,NaN,6,2
5,CLE,6/13/2021,1,669456,Shane Bieber,SP,Y,NaN,4,5,...,1.0,0,1.0,11.55,11000.0,CLE,v sea,NaN,2,6
6,ATL,6/13/2021,1,592767,Drew Smyly,SP,Y,NaN,1,7,...,1.0,9,1.0,19.05,6000.0,ATL,@ mia,NaN,6,4
7,MIA,6/13/2021,1,641154,Pablo Lopez,SP,Y,NaN,3,5,...,1.0,9,1.0,-2.65,8700.0,MIA,v atl,NaN,4,6
8,COL,6/13/2021,1,622608,Antonio Senzatela,SP,Y,NaN,4,4,...,1.0,9,1.0,5.50,6300.0,COL,@ cin,NaN,2,6
9,BAL,6/13/2021,1,669145,Bruce Zimmermann,SP,Y,NaN,2,7,...,1.0,0,1.0,7.35,6600.0,BAL,@ tam,NaN,1,7
